**Connecting to API**

In [ ]:
!pip install spotipy


# Import of libraries

import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import requests
from bs4 import BeautifulSoup
import shutil
from PIL import Image, UnidentifiedImageError
from IPython.core.display import display,HTML
import numpy as np
import os

# Enter credentials from Spotify API

cid ='5905590183fc4de6a5fb46f4e59adbce'
csecret ='37f2ab3df0e74d9b9accb78bb07009d6'

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=csecret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

**Forming dataset**

In [ ]:
# Playlists ids

play_lists = ["2NfTM2df5tHVUquNwet0yB", "0jH8HQYOmVYxHmb7prkgLb",
             "07iqzKaeKn1NsEScOzTmBU", "1G8IpkZKobrIlXcVPoSIuf",
             "5S8SJdl1BDc0ugpkEvFsIL", "0z085jfTrSXxhFUyKKExxJ", 
             "7I6XVNQVayB46Zzh4GNPoU", "3WWv2APRYe9jR8L1YnK1Up",
             "29isGIzk5nnq9J9fnjDgD4", "6Vr7VftW8axlF8FzSjB5ff",
             "5xqpyfZyS1DVysoevdVyEn", "5OA5UAo3z5QnEzdiugiRBd",
             "4AeheqQLQs0kZMRpMRVkHI"]


# Function to extract data from a playlist 
# (thank to sfnxboy)


def get_playlist_data(username, playlist_id):
    results = sp.user_playlist_tracks(username,playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    results = tracks    

    playlist_tracks_id = []
    playlist_tracks_titles = []
    playlist_tracks_artists = []
    playlist_artist_ids = []
    playlist_tracks_first_artists = []
    playlist_tracks_first_release_date = []
    playlist_tracks_popularity = []

    for i in range(len(results)):
        
        if i == 0:
            playlist_tracks_id = results[i]['track']['id']
            playlist_tracks_titles = results[i]['track']['name']
            playlist_tracks_first_release_date = results[i]['track']['album']['release_date']
            playlist_tracks_popularity = results[i]['track']['popularity']
            
            artist_list = []
            for artist in results[i]['track']['artists']:
                artist_list.append(artist['name'])
                
            playlist_tracks_artists = artist_list
            

            artist_id = []
            for artist in results[i]['track']['artists']:
                artist_id.append(artist['id'])

            playlist_artist_ids = artist_id
            

            features = sp.audio_features(playlist_tracks_id)
            features_df = pd.DataFrame(data=features, columns=features[0].keys())
            features_df['title'] = playlist_tracks_titles
            features_df['all_artists'] = [playlist_tracks_artists]
            features_df['artist_id'] = [playlist_artist_ids]
            features_df['popularity'] = playlist_tracks_popularity
            features_df['release_date'] = playlist_tracks_first_release_date
            features_df = features_df[['id', 'title', 'all_artists', 'artist_id', 'popularity', 'release_date',
                                       'danceability', 'energy', 'key', 'loudness',
                                       'mode', 'acousticness', 'instrumentalness',
                                       'liveness', 'valence', 'tempo',
                                       'duration_ms', 'time_signature']]
            continue
        else:
            try:
                playlist_tracks_id = results[i]['track']['id']
                playlist_tracks_titles = results[i]['track']['name']
                playlist_tracks_first_release_date = results[i]['track']['album']['release_date']
                playlist_tracks_popularity = results[i]['track']['popularity']
                artist_list = []
                for artist in results[i]['track']['artists']:
                    artist_list.append(artist['name'])
                playlist_tracks_artists = artist_list

                artist_id = []
                for artist in results[i]['track']['artists']:
                    artist_id.append(artist['id'])
                playlist_artist_ids = artist_id

                features = sp.audio_features(playlist_tracks_id)
                new_row = {'id':[playlist_tracks_id],
               'title':[playlist_tracks_titles],
               'all_artists':[playlist_tracks_artists],
               'artist_id':[playlist_artist_ids],
               'popularity':[playlist_tracks_popularity],
               'release_date':[playlist_tracks_first_release_date],
               'danceability':[features[0]['danceability']],
               'energy':[features[0]['energy']],
               'key':[features[0]['key']],
               'loudness':[features[0]['loudness']],
               'mode':[features[0]['mode']],
               'acousticness':[features[0]['acousticness']],
               'instrumentalness':[features[0]['instrumentalness']],
               'liveness':[features[0]['liveness']],
               'valence':[features[0]['valence']],
               'tempo':[features[0]['tempo']],
               'duration_ms':[features[0]['duration_ms']],
               'time_signature':[features[0]['time_signature']]
               }

                dfs = [features_df, pd.DataFrame(new_row)]
                features_df = pd.concat(dfs, ignore_index = True)
                
            except:
                continue

    features_df['all_artists'] = features_df['all_artists'].apply(lambda x: ','.join(map(str, x)))
    features_df['artist_id'] = features_df['artist_id'].apply(lambda x: ','.join(map(str, x)))
    
    return features_df

def get_track_genres(df):
    """
    Given a DataFrame `union_df` with a column named `artist_id`, returns a new DataFrame
    with an additional column `genres` containing a comma-separated list of genres for each
    track specified by the artist IDs in the `artist_id` column.

    Parameters:
    - df (pandas.DataFrame): the input DataFrame

    Returns:
    - df (pandas.DataFrame): df with an additional column `genres`
    """

    artist_ids = df.artist_id.to_list()
    genres = []
    for id in artist_ids:
        artist_ids_split = id.split(',')
        track_genres = []
        for element in artist_ids_split:
            genre = sp.artist(element)['genres']
            track_genres.extend(genre)
            track_genres = list(dict.fromkeys(track_genres))
        genres.append(track_genres)

    
    df['genres'] = genres
    df['genres'] = df['genres'].apply(', '.join)
    return df

def get_spotify_covers(df):
    """
    Given a DataFrame `df` with a column named `id`, returns a new DataFrame
    with an additional column `cover` containing the URL for the album cover image
    on Spotify for each track specified by `id`.

    If a cover image could not be found for a track, the value for that track in the
    `cover` column will be '-'

    Parameters:
    - df (pandas.DataFrame): the input DataFrame

    Returns:
    - df (pandas.DataFrame): df with an additional column `cover`
    """
    id_list = df.id.to_list()
    covers = []
    for id in id_list:
        try:
            html_page = requests.get(f'https://open.spotify.com/track/{id}')
            soup = BeautifulSoup(html_page.content, 'html.parser')
            images = soup.findAll('img', alt='')
            source = images[0].attrs['src']
            covers.append(source)
        except IndexError:
            covers.append('-')

    df['cover'] = covers
    return df

def download_covers(df, output_dir):
    for index, row in df.iterrows():
        source = row['cover']
        track_id = row['id']
        if source == '-':
            continue
        r = requests.get(source, stream=True)
        if r.status_code == 200:
            with open(f'{output_dir}/{track_id}.jpg', 'wb') as f:
                r.raw.decode_content = True
                shutil.copyfileobj(r.raw, f)

def extract_colors(df, folder_path):
    colors_df = pd.DataFrame(columns=['filename', 'blue', 'black', 
                                      'white', 'purple', 'green', 'yellow', 
                                      'red', 'orange', 'gray', 'pink'])

    for file_name in os.listdir(folder_path):

        try:
            # Open the image file
            image = Image.open(os.path.join(folder_path, file_name))

            # Define the colors to extract
            colors = {
                'blue': (51, 71, 150),
                'black': (27, 27, 27),
                'white': (255, 255, 255),
                'purple': (123, 31, 162),
                'green': (86, 136, 17),
                'yellow': (243, 195, 0),
                'red': (174, 2, 2),
                'orange': (238, 130, 2),
                'gray': (182, 182, 182),
                'pink': (224, 80, 182)
            }

            # Convert the colors dictionary to a numpy array for processing
            color_values = np.array(list(colors.values()))

            # Convert the image to a numpy array for processing
            image_array = np.array(image)

            # Reshape the image array into a long 2D array of pixels
            pixel_array = image_array.reshape(-1, 3)

            # Find the nearest color in the colors array for each pixel
            color_indices = np.argmin(np.sqrt(np.sum((pixel_array[:, np.newaxis, :] - color_values) ** 2, axis=2)), axis=1)

            # Count the number of pixels for each color
            color_counts = np.bincount(color_indices, minlength=len(colors))

            # Calculate the percentages of each color
            color_percentages = color_counts / np.sum(color_counts)

            # Create a dataframe with the results
            result_df = pd.DataFrame({
                'filename': file_name,
                **{color_name: [color_percentage] for color_name, color_percentage in zip(colors.keys(), color_percentages)}
            })

            colors_df = pd.concat([colors_df, result_df], ignore_index=True)
        except UnidentifiedImageError:
            print(f"Error opening file: {file_name}")
            continue

    colors_df['filename'] = colors_df['filename'].str.replace('.jpg', '')
    result_df = pd.merge(df, colors_df, left_on='id', right_on = 'filename', how='left')
    result_df = result_df.drop('filename', axis=1)

    return result_df

def display_dataframe(df, num_rows=100):
    def path_to_image_html(path):
        return '<img src="'+ path + '" width="60" >'
    
    return HTML(df.head(num_rows).to_html(escape=False, formatters=dict(cover=path_to_image_html)))

def main(playlist):
  main_df = pd.DataFrame()
  for item in playlist:
      playlist_df = get_playlist_data('dmitry_gama', item)
      main_df = pd.concat([main_df, playlist_df], ignore_index = True)
  main_df.drop_duplicates(subset = 'id', inplace = True, ignore_index=True)
  main_df = get_track_genres(main_df)
  main_df = get_spotify_covers(main_df)
  download_covers(main_df, '/content/track_covers')
  main_df = extract_colors(main_df, '/content/track_covers')
  return main_df

union_df = main(play_lists)
display_dataframe(union_df, num_rows=100)

,id,title,all_artists,artist_id,popularity,release_date,danceability,energy,key,loudness,mode,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,genres,cover,blue,black,white,purple,green,yellow,red,orange,gray,pink
0,6H7RUDjmjiGV4I9hE24Kk5,drivers license,jxdn,6Y64EaNqpqcZYTgs4c76gF,54,2021-01-19,0.598,0.787,10,-4.695,1,0.017500,0.000000,0.0887,0.2980,144.021,237644,4,"pop punk, social media pop",,0.018133,0.886756,0.041644,0.000167,0.000144,0.000000,0.016322,0.000456,0.036356,0.000022
1,0we7ShV1o6cPTFjxOADPbC,Outcast,NF,6fOMl44jA4Sp5b9PpYCkzz,58,2017-10-06,0.739,0.751,2,-7.161,1,0.165000,0.000000,0.0599,0.3210,130.176,325720,4,"hip hop, pop rap, rap",,0.151322,0.755044,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.093633,0.000000
2,6V81K1OcvYgHEnCA6A2Qx9,Start a Riot,"Duckwrth,Shaboozey","6I3MElirhT5t6Kf7p0hGk9,3y2cIKLjiOlp1Np37WiUdH",64,2018-12-14,0.728,0.727,2,-3.585,1,0.012800,0.000000,0.0882,0.8900,143.144,171467,4,"alternative r&b, escape room, indie r&b, pop rap, underground hip hop, viral rap",,0.008078,0.771056,0.104356,0.006389,0.000000,0.000000,0.066889,0.000167,0.040389,0.002678
3,37PJvlIfYWqEX6fgjHlnTf,WTF Do I Know,Miley Cyrus,5YGY8feqx7naU7z4HrwZM6,66,2020-11-27,0.564,0.849,4,-2.619,0,0.002550,0.000000,0.2690,0.6340,157.905,171800,4,pop,,0.008733,0.631933,0.005844,0.084622,0.001322,0.000000,0.025667,0.000000,0.004433,0.237444
4,3b2ngmuuJP2XCUUwZvK9WQ,Every Window Is A Mirror,Joywave,1UfzhwcOR4yfX7yHTPfC9m,33,2021-05-07,0.503,0.667,0,-4.166,1,0.001040,0.000438,0.0991,0.6800,105.033,173227,4,"indie poptimism, indietronica, modern alternative rock, modern rock, rochester ny indie, rock, shimmer pop, vapor soul",,0.172956,0.308333,0.310211,0.000456,0.000033,0.000000,0.000000,0.000000,0.208011,0.000000
5,1FjHxl8B28sXj7eVfxdST4,Better With You,Michl,0qG3lxHmrUeKzL1BJJ7IBN,42,2018-03-28,0.784,0.437,5,-7.868,1,0.265000,0.000070,0.1260,0.4700,90.003,196018,4,NaN,,0.058000,0.359489,0.306444,0.005133,0.007089,0.000000,0.000589,0.007578,0.254333,0.001344
6,0MMhDrqHLt67z0B1jlecqe,summer is gone,Omri,2Lku5fIu2jo19qXhz82Qio,15,2020-09-24,0.406,0.401,8,-8.440,1,0.400000,0.000000,0.1190,0.1390,112.880,162212,3,NaN,,0.215700,0.587822,0.069278,0.002978,0.001511,0.000000,0.000000,0.000000,0.122333,0.000378
7,4Sfa7hdVkqlM8UW5LsSY3F,Take a Walk,Passion Pit,7gjAu1qr5C2grXeQFFOGeh,65,2012-07-23,0.566,0.755,11,-5.526,1,0.033800,0.000000,0.3150,0.4450,101.006,264493,4,"alternative dance, indie rock, indietronica, modern rock, new rave, stomp and holler",,0.000611,0.002222,0.262367,0.018911,0.001000,0.000000,0.005867,0.000000,0.370944,0.338078
8,6WcSm79HcRk5Twgv7G4wFM,Unspoken,"Daktyl,MOONZz","2YrfEkWJvn1wue6JqekigJ,7cTaYwNT1P49mASgKULKab",30,2020-10-23,0.383,0.731,1,-6.703,0,0.755000,0.343000,0.1250,0.2250,106.105,204082,3,"indie electropop, indie poptimism, pop edm, vapor soul, vapor twitch",,0.221000,0.248456,0.000722,0.126933,0.000522,0.000000,0.001689,0.000300,0.382100,0.018278
9,0pyJOAI0NXUSwJlw9z1e0Y,Object,"Alter.,Barren Gates","6QbSwCeNaIhbfp1jpZ02nZ,7eMykQ9NjmtA5FnMarEG2F",48,2019-08-09,0.629,0.880,11,-4.380,0,0.014000,0.000404,0.0944,0.6230,140.029,125143,4,"bass trap, gaming edm, traprun",,0.065433,0.592022,0.041611,0.003278,0.005478,0.000000,0.178367,0.001067,0.096789,0.015956
